In [1]:
import numpy as np
import numpy.random as rnd
import theano
import theano.tensor as T
import lasagne
import time
import os
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN not available)


#### Все места, где нужно дописать код отмечены TODO.

## Считывание и подготовка данных.

In [38]:
# Считываем данные: каждый класс лежит в своем csv файле. 
male = pd.read_csv('male.csv',header = None)[0]
female = pd.read_csv('female.csv',header = None)[0]

y = np.hstack((np.zeros(len(male)),np.ones(len(female))))
data = list(male)
data.extend(list(female))

In [39]:
# Для дальнейшей работы нам понадобится словарь символов + 
# мы не будем различать строчные и прописные буквы + 
# у нас все последовательности разной длины и нам нужно понимать, какова макимальная длина
MAX_LEN = 0
chars = set()
for i in xrange(len(data)):
    data[i] = data[i].lower()
    MAX_LEN = max(MAX_LEN,len(data[i]))
    chars = chars.union(set(data[i][:]))
    
chars = list(chars)
char_to_id = { ch:id for id,ch in enumerate(chars) }
id_to_char = { id:ch for id,ch in enumerate(chars) }

VOCAB_SIZE = len(chars)

In [40]:
# Разделим выборку на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=42)

In [41]:
def data2format(data, labels):
    """Функция преобразует выбоку данных в формат, подходящий для подачи в нейронную сеть.
    
    data - список строк (пример - X_train)
    labels - вектор меток для строк из data (пример - y_train)
    
    Дальше за N обозначается число строк в data
    
    Вернуть нужно словарь со следующими элементами:
    x - матрица размера [N, MAX_LEN], в которой каждая строка соответствует строке в data:
        к строке прибавляется символы начала и конца строки, 
        после чего вся строка кодируется с помощью char_to_id.
        Недостающие элементы в конце коротких строк заполняются нулями
    mask - бинарная матрица размера [N, MAX_LEN]:
        единица говорит о том, что в соответствующем элементе x стоит значащий символ
        ноль говорит о том, что соответствующий элемент x не несет информации 
        (те самые нули, которые просто дополняют строки до MAX_LEN)
    y - вектор длины N с метками
    
    """
    
    # TODO
    return {'x':x,'mask': mask,'y': np.array(y,dtype='int32')}

In [42]:
train_data = data2format(X_train,y_train)
test_data = data2format(X_test,y_test)

## Вспомогательные функции

In [43]:
# Необходимые константы
NUM_EPOCHS = 20
BATCH_SIZE = 100
SEQ_LEN = 20
LEARNING_RATE = 0.01
GRAD_CLIP = 100

In [44]:
# Технические вещи

# Вспомогательная функция для запаковки результата обучения 
def pack(train_err, train_acc, test_err,test_acc, network, inp, inp_mask,target,train_fn, test_fn):
    return {'train_err':train_err, 
        'train_acc':train_acc, 
        'test_err':test_err, 
        'test_acc':test_acc, 
        'network':network,
        'inp':inp, 
        'inp_mask':inp_mask,
        'target':target,
        'train_fn':train_fn, 
        'test_fn':test_fn
           } 

## Нейронная сеть

В простейшем случае мы будем использовать сеть, которая считывает входную последовательность, и выдает результат только в самом конце.

In [45]:
def build_network(input_var=None, input_mask=None, emb_size = 40, n_hidden = 100):
    """Функция строит простейшую рекуррентную сеть, которая состоит из следующих слоев:
    
    1. Входной слой размера [BATCH_SIZE, MAX_LEN]. 
    2. Embedding для перевода кодировки символов в нормальное представление: VOCAB_SIZE -> emb_size
    3. Входной слой для маски размера [BATCH_SIZE, MAX_LEN]
    4. Рекуррентный слой c n_hidden элементов на скрытом слое:
        * этому слою кроме обычного входа нужно подавать еще и mask для правильной работы 
            с последовательностями разной длины
        * из этого слоя нам нужно только выход в последний момент времени, 
            его можно извлечь с помощью only_return_final
    5. Обычный полносвязный слой для бинарной классификации с sigmoid в качестве нелинейности
    
    Чтобы в дальнейшем мы могли запускать сеть, например, на одной последовательности, 
    для входного слоя и маски стоит прописывать shape=(None, None)
    """
    # TODO
    
    return network

In [46]:
def train(train_data, test_data, emb_size, n_hidden, show = False):
    """Функция обучает нейросеть по данным train_data + контролирует процесс по качеству на test_data
    Следует обратить внимание на следующее:
    1. Сеть будем учить NUM_EPOCHS эпох, в каждой из столько батчей, сколько есть в train_data
    2. Перед каждой эпохой желательно перемешивать данные с помощью shuffle
    3. Для того, чтобы следить за процессом обучения будем считать средний loss и 
        среднюю точность классификации на всех батчах трейна и теста и сохранять жти данные 
        в соответствующие массивы. 
    4. Перед тем, как делать шаг по градиенту, будем ограничивать градиент по норме
        с помощью функции lasagne.updates.total_norm_constraint с ограничением на норму GRAD_CLIP
    
    """
    print("Prepare data ...")
    train_x, train_mask, train_y = train_data['x'], train_data['mask'],train_data['y']
    test_x, test_mask, test_y = test_data['x'], test_data['mask'],test_data['y']
    
    train_size = len(train_y)
    test_size = len(test_y)
    num_train_batches = train_size / BATCH_SIZE
    num_test_batches = test_size / BATCH_SIZE
    train_err=np.zeros(NUM_EPOCHS)
    train_acc=np.zeros(NUM_EPOCHS)
    test_err=np.zeros(NUM_EPOCHS)
    test_acc=np.zeros(NUM_EPOCHS)
        
    print("Building network ...")
    # TODO
    print("The network has {} params".format(lasagne.layers.count_params(network)))
    
    # Функции для loss, updates, train_fn и logprobs_fn
    # В качетсве loss стоит взять обычную бинарную cross-entropy
    # Для более устойчивого вычисления loss стоит обрезать предсказание 
    # перед подсчетом loss: T.clip(prediction,1e-7,1-1e-7)
    # В качестве метода оптимизации рекомендуется использовать adam
    
    # TODO
    
    print("Training ...")
    for epoch in xrange(NUM_EPOCHS):
        start_time = time.time()
        # TODO

        print("Epoch {} \t loss / accuracy test = {:.4f}, {:.4f} \t train = {:.4f}, {:.4f} \t time = {:.2f}s".
              format(epoch, test_err[epoch],test_acc[epoch], 
                     train_err[epoch],  train_acc[epoch],time.time() - start_time))
             
    return pack(train_err, train_acc, test_err, test_acc, network, inp, inp_mask, target, train_fn, test_fn)

Перед тем, как запускать обучение большой сети на большое число эпох, проверьте, что маленькая сеть выдает вменяемые результаты: качество больше 50%.

In [47]:
model = train(train_data, test_data, 30, 400)

Prepare data ...
Building network ...
The network has 174071 params
Training ...
Epoch 0 	 loss / accuracy test = 0.4417, 78.1739 	 train = 0.4343, 78.0545 	 time = 0.63s
Epoch 1 	 loss / accuracy test = 0.4322, 79.0870 	 train = 0.4265, 78.4727 	 time = 0.61s
Epoch 2 	 loss / accuracy test = 0.4198, 80.1304 	 train = 0.3976, 80.7636 	 time = 0.61s
Epoch 3 	 loss / accuracy test = 0.4240, 79.9565 	 train = 0.4001, 80.4909 	 time = 0.61s
Epoch 4 	 loss / accuracy test = 0.4241, 79.3043 	 train = 0.3799, 80.8909 	 time = 0.61s
Epoch 5 	 loss / accuracy test = 0.4179, 79.7391 	 train = 0.3680, 81.4727 	 time = 0.61s
Epoch 6 	 loss / accuracy test = 0.4188, 80.3913 	 train = 0.3610, 82.9273 	 time = 0.61s
Epoch 7 	 loss / accuracy test = 0.4368, 79.3043 	 train = 0.3863, 81.3455 	 time = 0.61s
Epoch 8 	 loss / accuracy test = 0.4314, 80.2174 	 train = 0.3356, 84.1091 	 time = 0.62s
Epoch 9 	 loss / accuracy test = 0.4143, 81.1739 	 train = 0.3335, 83.7273 	 time = 0.61s
Epoch 10 	 loss / a

## Посмотрим что из этого вышло

In [89]:
def predict(name, model):
    """Функция выдает предсказание обученной модели model для имени name.
    Предсказание - число из [0,1] - вероятность того, что имя женское
    """
    
    #TODO
    return y

In [78]:
dataset = set(data)

In [93]:
name = 'Yaroslav'
if name.lower() in dataset:
    print 'This name is in our dataset'
else:
    print 'This is a new name'
pred = predict(name, model)
if pred>=0.5:
    print "It's female name"
else:
    print "It's male name"
print pred

This is a new name
It's male name
[ 0.04011425]


In [90]:
name = 'Polina'
if name.lower() in dataset:
    print 'This name is in our dataset'
else:
    print 'This is a new name'
pred = predict(name, model)
if pred>=0.5:
    print "It's female name"
else:
    print "It's male name"
print pred

This is a new name
It's female name
[ 0.99993134]


## Дополнительные пункты

1. Обучение более сложной модели и контроль переобучения. Попробуйте подобрать хорошую модель RNN для данной задачи. Для этого проанализируйте качество работы модели в зависимости от ее размеров, попробуйте использовать многослойную сеть. Также нужно проконтролировать переобучение моделей. Для этого можно выделить тестовый кусок из текста и смотреть на то, как меняется loss на нем в процессе обучения. Если на графиках видно переобучение, то стоит добавить dropout слои в модель (обычный dropout до, между и после рекуррентных слоев). При использовании дропаута на стадии предсказания для нового объекта нужно ставить флаг deterministic=True.
2. Другая архитектура 1. Попробуйте использовать не только состоянию скрытых переменных в последний момент времени, а усреднение/максимум значений скрытых переменных во все моменты времени. Попробуйте двунаправленную сеть при таком подходе. 
3. Другая архитектура 2. Попробуйте использовать не только состоянию скрытых переменных в последний момент времени, а сумму значений скрытых переменных во все моменты времени с коэффициентами attention. Попробуйте двунаправленную сеть при таком подходе. Attention коэффициент для определенного момента времени может представлять собой просто линейную комбинацию значений скрытых переменных в этот момент времени с обучаемыми весами.
3. Визуализация. Попробуйте провизуализировать результаты. Например, для стандартной архитектуры можно посмотреть на изменение предсказания во времени: на каких элементах предсказание значительнее всего изменяется в сторону одного или другого класса? При использовании схемы из 2/3 пункта, можно смотреть на вклад каждого момента времени в результат. Так как после рекуррентного слоя у нас стоит просто линейный классификатор, то можно посмотреть, что выдает этот классификатор при применении к скрытым переменным в каждый момент времени. Таким образом выделяться те буквы, которые голосуют за один класс и те, которые голосуют за другой.
4. Batchnorm и Layernorm. Запрограммируйте RNN c layer normalization из статьи [Lei Ba et al., 2016]. Поэкспериментируйте с применением обычной batch normalization и layer normalization, сравните результаты.